In [1]:
import tensorflow as tf
print(tf.__version__)


2023-10-05 14:39:18.775120: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/rau/pythonProject1/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


2.14.0


In [3]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load data
tabular_data = pd.read_csv('/Users/rau/Desktop/Asa/nganjuk_df_clean.csv')
tabular_data['date'] = pd.to_datetime(tabular_data['date']).dt.strftime('%Y-%m-%d')

# Process images
image_folder = '/Users/rau/Desktop/Asa/NDVI_Stasiun Geofisika Nganjuk (3)'
image_files = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith('.png')]
image_dates = [os.path.splitext(os.path.basename(fname))[0] for fname in image_files]
image_path_dict = dict(zip(image_dates, image_files))
tabular_data['image_path'] = tabular_data['date'].apply(lambda date: image_path_dict.get(date))

# Model instantiation
model = VGG16(include_top=False, weights='imagenet', pooling='avg')

# Feature extraction
def extract_features(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    return features[0]

tabular_data['image_features'] = tabular_data['image_path'].apply(extract_features)
tabular_data['image_id'] = tabular_data['image_path'].apply(lambda path: os.path.splitext(os.path.basename(path))[0])

# Split data
X = list(tabular_data['image_features'])
y = tabular_data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Evaluate model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")


ModuleNotFoundError: No module named 'tensorflow.keras.applications.vgg16.preprocess_input'